<a href="https://colab.research.google.com/github/aubricot/object_detection_for_image_cropping/blob/master/train_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training YOLO in Darkflow to detect bats from EOL images
---
*Last Updated 3 February 2020*   
Use images and annotation files to train YOLO in Darkflow to detect bats from EOL images.

Datasets exported from [preprocessing.ipynb](https://colab.research.google.com/github/aubricot/object_detection_for_image_cropping/blob/master/preprocessing.ipynb) were converted to xml formatted annotation files before use in this notebook. Images were already downloaded to Google Drive in preprocessing.ipynb. Annotations should be uploaded to Google Drive before using this notebook. Exported detection results (json files) can be used to calculate model precision for comparison with Faster-RCNN and SSD models using [calculate_error_mAP.ipynb](https://colab.research.google.com/github/aubricot/object_detection_for_image_cropping/blob/master/calculate_error_mAP.ipynb). 

Notes:   
* For each 24 hour period on Google Colab, you have up to 12 hours of GPU access. Training the object detection model on bats took 30 hours split into 3 days.

* Make sure to set the runtime to Python 2 with GPU Hardware Accelerator.   

References:   
* [Official Darkflow training instructions](https://github.com/thtrieu/darkflow)   
* [Medium Blog on training using YOLO via Darkflow in Colab](https://medium.com/coinmonks/detecting-custom-objects-in-images-video-using-yolo-with-darkflow-1ff119fa002f)

## Installs
---

In [0]:
# Mount google drive to import/export files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
# Change to your working directory
%cd drive/My Drive/fall19_smithsonian_informatics/train

# Install libraries
# Make sure you are using Python 3.6
!python --version
!pip install tensorflow-gpu==1.15.0rc2
!pip install cython
!pip install opencv-python

In [0]:
# Download darkflow (the tensorflow implementation of YOLO)
import os
import pathlib
import shutil 

if os.path.exists("darkflow-master"):
  %cd darkflow-master/darkflow
  !pwd

elif not os.path.exists("darkflow-master"):
    !git clone --depth 1 https://github.com/thtrieu/darkflow.git
    # Compile darkflow
    %cd darkflow
    !python setup.py build_ext --inplace
    # Rename darkflow to darkflow-master to distinguish between folder names
    shutil.move('/content/drive/My Drive/fall19_smithsonian_informatics/train/darkflow', 
              '/content/drive/My Drive/fall19_smithsonian_informatics/train/darkflow-master')

# Change wd to darkflow-master
%cd ../

### Imports   
---

In [0]:
%cd darkflow-master

# For importing/exporting files, working with arrays, etc
from google.colab import files
import os
import pathlib
import imageio
import time
import csv
import urllib
import numpy as np
import pandas as pd

# For the actual object detection
from darkflow.net.build import TFNet

# For drawing onto and plotting the images
import matplotlib.pyplot as plt
import cv2
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

### Model Preparation (only need to run these once)
---   
For detailed instructions on training YOLO using a custom dataset, see the [Darkflow GitHub Repository](https://github.com/thtrieu/darkflow).

In [0]:
# Test installation, you should see an output with different parameters for flow
!python flow --h

In [0]:
# Upload yolo.weights, pre-trained weights file (for YOLO v2) from Google drive 
weights = 'bin/yolo'
weights_file = weights + '.weights'
if not os.path.exists('weights_file'): # why is this downloading when the path exists?
  !gdown --id 0B1tW_VtY7oniTnBYYWdqSHNGSUU
  !mkdir bin
  !mv yolo.weights bin

In [0]:
# Make new label file/overwrite existing labels.txt downloaded with darkflow
!echo "Chiroptera" > labels.txt

# Download model config file edited for training darkflow to identify bats (yolo-1c = yolo to identify 1 class)
config = 'yolo-1c'
mod_config = config + '.cfg'
if not os.path.exists('mod_config'):
  %cd train/darkflow-master/cfg
  !gdown --id 1bjt5Mqvf4AZSLNARgtgmZsfHZSyFj2yx

## Train the model
---

In [0]:
# Train yolo-1c using pre-trained weights at yolo.weights for basal layers, last layer will be trained from scracth to detect bats
# Change the dataset and annotation directories to your paths in GoogleDrive
%cd darkflow-master
!python flow --model cfg/yolo-1c.cfg --train --trainer adam --load bin/yolo.weights --gpu 0.8 --epoch 3000 --dataset "/content/drive/My Drive/fall19_smithsonian_informatics/train/images" --annotation "test/training/annotations"

In [0]:
# For training the model starting from specified checkpoint
# change --load 750 to the number of steps of your most recent checkpoint file
#--pbLoad .pb --metaLoad .meta
!python flow --load -1 --model cfg/yolo-1c.cfg --train --savepb --trainer adam --gpu 0.8 --epoch 3000 --dataset "/content/drive/My Drive/fall19_smithsonian_informatics/train/images" --annotation "test/training/annotations"

In [0]:
# Save the last checkpoint to protobuf file
!python flow --model cfg/yolo-1c.cfg --load -1 --savepb

In [0]:
# Export detection results as json files fo calculating mAP in calculate_error_mAP.ipynb
!python flow --pbLoad built_graph/yolo-1c.pb --gpu 0.8 --metaLoad built_graph/yolo-1c.meta --imgdir "/content/drive/My Drive/fall19_smithsonian_informatics/train/test_images" --json

## Load in test images and run them through the trained object detector
---

In [0]:
# For loading images into computer-readable format
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

# For drawing bounding boxes around detected objects on images
def boxing(image, predictions):
    newImage = np.copy(image)
    im_height, im_width, im_depth = image.shape

    # Oraganize results of object detection for plotting and export
    for result in predictions:
        xmin = result['topleft']['x']
        ymin = result['topleft']['y']

        xmax = result['bottomright']['x']
        ymax = result['bottomright']['y']

        confidence = result['confidence']
        label = result['label'] + " " + str(round(confidence, 3))

        # only show boxes that are above set confidence and for the label Chiroptera
        if confidence > 0 and result['label'] == 'Chiroptera' :
            # draw boxes on images
            fontScale = min(im_width,im_height)/(600)
            newImage = cv2.rectangle(newImage, (xmin, ymax), (xmax, ymin), (255, 0, 157), 3)
            newImage = cv2.putText(newImage, label, (xmin, ymax-5), cv2.FONT_HERSHEY_SIMPLEX, fontScale, (153, 255, 255), 5, cv2.LINE_AA)

    return newImage

# Define parameters for "flow"ing the images through the model
params = {
    'model': 'cfg/yolo-1c.cfg',
    'load': 'bin/yolo.weights',
    'gpu': 0.8,
    #'threshold': 0.1, 
    'pbLoad': 'built_graph/yolo-1c.pb', 
    'metaLoad': 'built_graph/yolo-1c.meta' 
}

# Run the model
tfnet = TFNet(params)

In [0]:
# Test trained model on test images
from PIL import Image

# Update path to your test images
PATH_TO_TEST_IMAGES_DIR = '/content/drive/My Drive/fall19_smithsonian_informatics/train/test_images'
names = os.listdir(PATH_TO_TEST_IMAGES_DIR)
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, name) for name in names]

# Loops through first 5 image urls from the text file
for im_num, im_path in enumerate(TEST_IMAGE_PATHS[:5], start=1):

    # Record inference time
    start_time = time.time()
    image = Image.open(im_path)
    image_np = load_image_into_numpy_array(image)
    # Detection
    result = tfnet.return_predict(image_np)
    end_time = time.time()
    # Draw boxes on images
    boxing(image_np, result)
  
    # If running detection on >50 images, do not display detection results
    # Instead run below command to track progress
    print('Detection complete in {} of 145 test images'.format(im_num))

    # Plot and show detection boxes on images
    # Hashtag out this portion if running detection on >50 images
    _, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(boxing(image_np, result))
    plt.title('{}) Inference time: {}'.format(im_num, format(end_time-start_time, '.2f')))
    #plt.close()